In [3]:
library (foreign)
library(sqldf)
library(DBI)
library(RPostgreSQL) #dbi
library(RODBC)
library(sp)
library(rgdal) #sp
library(rgeos)
library(maptools)
library(PBSmapping)
library(dplyr)
library(modes)
library(ggplot2)
library(stringr)

In [4]:
#SECRETARIADO
secretariado<- read.csv("/home/rosa/Descargas/BASE_DE_DATOS_ENVIPE_2017_en/secretariado.csv")

In [5]:
claveEntMun<-function(x) {
  estmun<-c()
  for (i in 1:length(CVEGEO))
  {
    largo<-nchar(CVEGEO[i])
    if(largo == 4){
      union<-paste("0",CVEGEO[i], sep= "")
      estmun[i]<-rbind(union) }
    else{
      estmun[i]<-rbind(CVEGEO[i]) }
    #if(largo == 1){replace("0",CVE_ENT, sep= "")}
  }
  return(estmun)
}

In [6]:
#Claves de delitos prioritarios: 
#Extorsión: PAText; Homicidio doloso: VIChod; Secuestro: LPEsec

#Se desagrega la base de datos para poder armar una clave concatenada con año,clave del municipio, clave del delito y tipo
#Postetiormente se arma un data.frame con este campo y todos los meses subsecuentes

attach(secretariado)
CVEGEO<-CVEGEOg
CVEGEO<-claveEntMun(CVEGEO) #CONVERSION A TEXTO DE CAMPO
concatena<-paste(year,CVEGEO,clave,tipo,sep = "-")
secretariado<-data.frame(concatena,ene,feb,mar,abr,may,jun,jul,ago,sep,oct,nov,dic)

In [7]:
#Se suman todas las denuncias de un mismo tipo de delito por cada municipio
s2<-aggregate(secretariado[2:13], by=list(concatena), FUN=sum)
#Una vez sumados se desagrega la clave concatenada para obtener los campos
desconca<-str_split_fixed(s2$Group.1,"-",4)
#Se arma un nuevo data frame para agregar a el campo que se fragmentó y los datos de los meses
s3<-data.frame(desconca,s2[2:13])
#Se hace un nuevo campo de concatennación cib clave del municipio, clave de delito y tipo
concatena<-paste(s3$X2,s3$X3,s3$X4,sep = "-")
#Se arma un nuevo data frame, agregando el campo de concatenacion
s3<-data.frame(concatena,s3)
#Se le dan nuevos nombres
colnames(s3)<-c("concatena","year","CVEGEO","clave","tipo","ene","feb","mar","abr","may","jun","jul","ago","sep","oct","nov","dic")

In [8]:
#Se obtienen conjuntos de datos por años
s3_2016<-subset(s3,s3$year ==2016 )
s3_2017<-subset(s3,s3$year ==2017 )
s3_2018<-subset(s3,s3$year ==2018 )

#Se hace el acomodo de los periodos JULIO 2016-2017 y JULIO 2017 - 2018. 
#Adicionalmente se calcula el total para ese periodo
#Por ultimo se desagrega el campo de concatenación
s3_1617<- merge(x = s3_2016[c("concatena","jul","ago","sep","oct","nov","dic")], y = s3_2017[c("concatena","ene","feb","mar","abr","may","jun")], by = "concatena", all = TRUE)
s3_1718<- merge(x = s3_2017[c("concatena","jul","ago","sep","oct","nov","dic")], y = s3_2018[c("concatena","ene","feb","mar","abr","may","jun")], by = "concatena", all = TRUE)
s3_1617[is.na(s3_1617)]<- 0
s3_1718[is.na(s3_1718)]<- 0
s3_1617$total<- rowSums(s3_1617[2:13])
s3_1718$total<- rowSums(s3_1718[2:13])
s3_1617<-data.frame(str_split_fixed(s3_1617$concatena,"-",3),s3_1617[14])
s3_1718<-data.frame(str_split_fixed(s3_1718$concatena,"-",3),s3_1718[14])


In [9]:
#Se obtienen tabulados por cada año, en el cual cada columna es un delito y el numero de denuncias que tienen. 
# En las filas se encuentran cada uno de los municipios ese año (el ejercicio es para ambos periodos de tiempo)
# JULIO 2016-2017 y JULIO 2017 - 2018

delitos1617<-data.frame(tapply(s3_1617$total,list(s3_1617$X1,s3_1617$X2),sum))
delitos1617<-data.frame(rownames(delitos1617),delitos1617)
delitos1617[is.na(delitos1617)]<- 0

delitos1718<-data.frame(tapply(s3_1718$total,list(s3_1718$X1,s3_1718$X2),sum))
delitos1718<-data.frame(rownames(delitos1718),delitos1718)
delitos1718[is.na(delitos1718)]<- 0


In [10]:
#Se tienen que obtener dos grupos de delitos
#(1) Delitos prioritarios: Establecidos por la base de Ana Alegre, los cuales son: 
#    Extorsión: PAText; Homicidio doloso: VIChod; Secuestro: LPEsec
#(2) Delitos reincidentes: o delitos con el mayor numero de denuncias, omitiendo los delitos prioritarios

#Para esta parte se decidió iniciar con los delitos reincidentes, para lo cual las bases de datos fuente 
#(delitos1617 y delitos1718 respectivamente) fueron copiadas y se les quito los delitos prioritarios

#Para elegir los tres delitos de mayor reincidencia, se tomaron los delitos con mas cantidad de denuncias en
#el periodo Julio 2017 - 2018, y una vez tomados estos se consultaron los valores que tuvieron estos mismos para el 
#periodo Julio 2016-2017.

# Finalmente con estos datos se obtuvo la variación entre estos dos periodos

In [11]:
#(1) Definir cual es el top de delitos con base en Julio 2017-2018

#Se arman dataframes omitiendo los delitos prioritarios
d2_1617<-data.frame(delitos1617$rownames.delitos1617.,delitos1617$FAMiof,delitos1617$FAMofa,delitos1617$FAMvfa,delitos1617$FAMvgt,delitos1617$LPEolp,delitos1617$LPErap,delitos1617$LPEtme,delitos1617$LSSabs,delitos1617$LSSacs,delitos1617$LSShos,delitos1617$LSSinc,delitos1617$LSSols,delitos1617$LSSveq,delitos1617$LSSvsi,delitos1617$OBJalm,delitos1617$OBJame,delitos1617$OBJcma,delitos1617$OBJdsp,delitos1617$OBJele,delitos1617$OBJevp,delitos1617$OBJfal,delitos1617$OBJfas,delitos1617$OBJnar,delitos1617$OBJofu,delitos1617$PATabc,delitos1617$PATdap,delitos1617$PATdes,delitos1617$PATfra,delitos1617$PATopa,delitos1617$PATrob,delitos1617$SOCcom,delitos1617$SOCoso,delitos1617$SOCtpe,delitos1617$VICabo,delitos1617$VICfem,delitos1617$VIChom,delitos1617$VICles,delitos1617$VICoic)
colnames(d2_1617)<-c("conca","FAMiof","FAMofa","FAMvfa","FAMvgt","LPEolp","LPErap","LPEtme","LSSabs","LSSacs","LSShos","LSSinc","LSSols","LSSveq","LSSvsi","OBJalm","OBJame","OBJcma","OBJdsp","OBJele","OBJevp","OBJfal","OBJfas","OBJnar","OBJofu","PATabc","PATdap","PATdes","PATfra","PATopa","PATrob","SOCcom","SOCoso","SOCtpe","VICabo","VICfem","VIChom","VICles","VICoic"  )
d2_1718<-data.frame(delitos1718$rownames.delitos1718.,delitos1718$FAMiof,delitos1718$FAMofa,delitos1718$FAMvfa,delitos1718$FAMvgt,delitos1718$LPEolp,delitos1718$LPErap,delitos1718$LPEtme,delitos1718$LSSabs,delitos1718$LSSacs,delitos1718$LSShos,delitos1718$LSSinc,delitos1718$LSSols,delitos1718$LSSveq,delitos1718$LSSvsi,delitos1718$OBJalm,delitos1718$OBJame,delitos1718$OBJcma,delitos1718$OBJdsp,delitos1718$OBJele,delitos1718$OBJevp,delitos1718$OBJfal,delitos1718$OBJfas,delitos1718$OBJnar,delitos1718$OBJofu,delitos1718$PATabc,delitos1718$PATdap,delitos1718$PATdes,delitos1718$PATfra,delitos1718$PATopa,delitos1718$PATrob,delitos1718$SOCcom,delitos1718$SOCoso,delitos1718$SOCtpe,delitos1718$VICabo,delitos1718$VICfem,delitos1718$VIChom,delitos1718$VICles,delitos1718$VICoic)
colnames(d2_1718)<-c("conca","FAMiof","FAMofa","FAMvfa","FAMvgt","LPEolp","LPErap","LPEtme","LSSabs","LSSacs","LSShos","LSSinc","LSSols","LSSveq","LSSvsi","OBJalm","OBJame","OBJcma","OBJdsp","OBJele","OBJevp","OBJfal","OBJfas","OBJnar","OBJofu","PATabc","PATdap","PATdes","PATfra","PATopa","PATrob","SOCcom","SOCoso","SOCtpe","VICabo","VICfem","VIChom","VICles","VICoic"  )

#Inicia el loop para obtener los tres mas frecuentes
#Funcionamiento: Se obtiene la fila correspondiente a un municipio, de la cual se obtienen la clave del delito maximo,
#                el valor del maximo, la posición del maximo; Se agregan en un texto para acumular el valor de cada
#                municipio; El valor maximo se vuelve cero para que en el proximo loop se detecte el segundo con mayor
#                cantidad; Finalmente esta cadena se desfragmenta para obtener un data frame con los valores ya mencionados
topdelitos<-data.frame("1","2","3","4","5","6","7")
for (i in 1:nrow(d2_1718))
{
  municipio<-d2_1718[i,]
  td<-data.frame(1,2,3)
  for(j in 1:3)
  {
    top<-data.frame(max(municipio[2:39]),which.max(municipio[2:39])) #which.max da la posición delelemento pedido
    top<-data.frame(rownames(top),top)
    colnames(top)<-c("cdelito","denuncias","posicion")
    top<-paste(top$cdelito,top$denuncias,top$posicion,sep="-")
    td[j]<-cbind(top)
    municipio[which.max(municipio[2:39])+1]<-0 #Vuelve cero la cantidad mayor
  }
  td4<-paste(municipio$conca,td$X1,td$X2,td$X3,sep="-")
  topdelitos[i]<-rbind(td4)
}
topdelitos<-data.frame(str_split_fixed(topdelitos,"-",10))
colnames(topdelitos)<-c("conca","del1","del1Num","del1Pos","del2","del2NUm","del2Pos","del3","del3NUm","del3Pos")

In [12]:
#(2) Obtener los valores del top3 de delitos para el periodo Julio 2016 - 2017

#Se arma un nuevo dataframe con el valor de la clave del municipio, y las tres posiciones de los delitos maximos
top<-data.frame(topdelitos$conca,topdelitos$del1Pos,topdelitos$del2Pos,topdelitos$del3Pos)
colnames(top)<-c("conca","del1Pos","del2Pos","del3Pos")

#Los municipios tanto de las posiciones como de las denuncias Julio 2016 - 2017 deben tener el mismo numero de municipios
#ya que de otra forma no puede compararse entre si. Este filtro se elabora por medio de dos uniones y así obtener
#los municipios que se encuentran en ambas bases
top<-merge(x = d2_1617[1], y = top , by = "conca", all = FALSE)
d2_1617_top<-merge(x = d2_1617, y = top[1] , by = "conca", all = FALSE)

#Inicia el loop para obtener los valores del top3
#Funcionamiento: El loop itera en cada uno de los municipios. Por ello se obtiene la fila de cada tabla (posiciones, Julio16-17)
#                por cada municipio y una vez ehecho esto, se obtienen los valores de los tres maximos consultando en
#                la base de denuncias, la posición que le corresponde; Posteriormente se colapsa en un texto para su
#                almacenamiento y una vez obtenidos todos los municipios se desfragmenta; Una vez que termina el loop
#                se une al periodo Julio1718 y se ordena
topdelitos1617<-data.frame("1","2","3","4","5","6","7")
for (i in 1:nrow(d2_1617_top))
{
  municipio<-d2_1617_top[i,]
  td<-top[i,]
    
  t1<-municipio[as.numeric(as.character(droplevels.factor(td$del1Pos)))+1]
  t2<-municipio[as.numeric(as.character(droplevels.factor(td$del2Pos)))+1]
  t3<-municipio[as.numeric(as.character(droplevels.factor(td$del3Pos)))+1]  
  
  j<-paste(municipio$conca,t1,t2,t3,sep="-")
  topdelitos1617[i]<-rbind(j)
}
topdelitos1617<-data.frame(str_split_fixed(topdelitos1617,"-",4))
colnames(topdelitos1617)<-c("conca","del1Num","del2NUm","del3NUm")

secret_final<-merge(x = topdelitos, y = topdelitos1617 , by = "conca", all = TRUE)
secret_final<-data.frame(secret_final$conca,secret_final$del1,secret_final$del1Num.y,secret_final$del1Num.x,secret_final$del2,secret_final$del2NUm.y,secret_final$del2NUm.x,secret_final$del3,secret_final$del3NUm.y,secret_final$del3NUm.x)
colnames(secret_final)<-c("conca","d1","d1_1617","d1_1718","d2","d2_1617","d2_1718","d3","d3_1617","d3_1718")


In [13]:
#(3) Delitos prioritarios

#NOTA: El homicidio doloso se considera delito prioritario, sin embargo en la base del secretariado, este se especifica 
#      hasta el subtipo (es decir lo "doloso"), por ello fue necesario modificar la clave al nivel de tipo, diferenciando
#      del homicidio culposo cambiando su abreviatura de "hom" a "hod" de forma manual.

#Al igual que el top3 de delitos, se necesitan ambos periodos JULIO 2016-2017 y JULIO 2017 - 2018, por ello se utilizan
#las bases delitos1617 y delitos1718

#Se obtienen los datos para ambis periodos, se unen y se arregla la base resultante
prioritarios1617<-data.frame(delitos1617$rownames.delitos1617.,delitos1617$PAText,delitos1617$VIChod,delitos1617$LPEsec)
colnames(prioritarios1617)<-c("conca","PAText1617","VIChod1617","LPEsec1617")
prioritarios1718<-data.frame(delitos1718$rownames.delitos1718.,delitos1718$PAText,delitos1718$VIChod,delitos1718$LPEsec)
colnames(prioritarios1718)<-c("conca","PAText1718","VIChod1718","LPEsec1718")
prioritarios<-merge(x = prioritarios1617, y = prioritarios1718 , by = "conca", all = TRUE)
prioritarios<-data.frame(prioritarios$conca,prioritarios$PAText1617,prioritarios$PAText1718,prioritarios$VIChod1617,prioritarios$VIChod1718,prioritarios$LPEsec1617,prioritarios$LPEsec1718)
colnames(prioritarios)<-c("conca","PAText1617","PAText1718","VIChod1617","VIChod1718","LPEsec1617","LPEsec1718")

#Finalmente se une a la base final del secretariado
secret_final<-merge(x = prioritarios, y = secret_final , by = "conca", all = TRUE)

In [ ]:
#(4) Calculo de Variaciones para top3 y delitos prioritarios

#Se realizan los calculos. Debido a que los datos cuentan con niveles, es necesario convertirlos a enteros simples
secret_final$PAText_var<-round(((as.numeric(as.character(droplevels.factor(secret_final$PAText1718))) - as.numeric(as.character(droplevels.factor(secret_final$PAText1617)))) * 100) / as.numeric(as.character(droplevels.factor(secret_final$PAText1617))),1)
secret_final$VIChod_var<-round(((as.numeric(as.character(droplevels.factor(secret_final$VIChod1718))) - as.numeric(as.character(droplevels.factor(secret_final$VIChod1617)))) * 100) / as.numeric(as.character(droplevels.factor(secret_final$VIChod1617))),1)
secret_final$LPEsec_var<-round(((as.numeric(as.character(droplevels.factor(secret_final$LPEsec1718))) - as.numeric(as.character(droplevels.factor(secret_final$LPEsec1617)))) * 100) / as.numeric(as.character(droplevels.factor(secret_final$LPEsec1617))),1)
secret_final$d1_var<-round(((as.numeric(as.character(droplevels.factor(secret_final$d1_1718))) - as.numeric(as.character(droplevels.factor(secret_final$d1_1617)))) * 100) / as.numeric(as.character(droplevels.factor(secret_final$d1_1617))),1)
secret_final$d2_var<-round(((as.numeric(as.character(droplevels.factor(secret_final$d2_1718))) - as.numeric(as.character(droplevels.factor(secret_final$d2_1617)))) * 100) / as.numeric(as.character(droplevels.factor(secret_final$d2_1617))),1)
secret_final$d3_var<-round(((as.numeric(as.character(droplevels.factor(secret_final$d3_1718))) - as.numeric(as.character(droplevels.factor(secret_final$d3_1617)))) * 100) / as.numeric(as.character(droplevels.factor(secret_final$d3_1617))),1)

#se arregla la base final
secret_final<-data.frame(secret_final$conca,secret_final$PAText1617,secret_final$PAText1718,secret_final$PAText_var,secret_final$VIChod1617,secret_final$VIChod1718,secret_final$VIChod_var,secret_final$LPEsec1617,secret_final$LPEsec1718,secret_final$LPEsec_var,secret_final$d1,secret_final$d1_1617,secret_final$d1_1718,secret_final$d1_var,secret_final$d2,secret_final$d2_1617,secret_final$d2_1718,secret_final$d2_var,secret_final$d3,secret_final$d3_1617,secret_final$d3_1718,secret_final$d3_var)
colnames(secret_final)<-c("conca","PAText1617","PAText1718","PATextVar","VIChod1617","VIChod1718","VIChodVar","LPEsec1617","LPEsec1718","LPEsecVar","d1","d1_1617","d1_1718","d1Var","d2","d2_1617","d2_1718","d2Var","d3","d3_1617","d3_1718","d3Var")
